# Sheet 07

In [ ]:
from numpy.typing import NDArray  # trying to be typesafe
import numpy as np  # needed all over the place
import scipy
from scipy.stats import skew
from IPython.display import HTML # in line animations

import os   # file and file path

from itertools import count
import pandas as pd
import matplotlib.pyplot as plt  # for plots

from matplotlib import colors  # not quite sure what fore
from matplotlib.ticker import PercentFormatter  # also not sure, maybe animation..?
from matplotlib.animation import FuncAnimation, FFMpegWriter  # for animations

##### Markov Chain - Monte Carlo  - Metropolis importance Sampling

In [ ]:
def initiate_positions_on_grid(
            num_particles: int,
            box: tuple [int,int],

    ) -> NDArray[np.float32]:
    """
    Initializes 2D positions on a Grid with even spacing.

    Parameters:
        
        n_particles:(int)
                -> number of particles
        
        box:(tuple[number,number]) 
                -> box size (in [nm])
        
    Return:
        (NDArray[float, shpae=(dimension, number of particles)])
                -> list of vectors of positons that are aranged on a grid
    """
    grid_sections = int(np.ceil(np.sqrt(num_particles)))  # find the number of colums & rows
    # even spacing
    x_spacing = box[0]/grid_sections 
    y_spacing = box[1]/grid_sections
    print("x_spacing:",x_spacing)
    print("y_spacing:",y_spacing)
    # makes grid coordinates
    x, y= np.meshgrid(
        np.arange(grid_sections) * x_spacing, 
        np.arange(grid_sections) * y_spacing
    )        # constants
    x= x.flatten()[:num_particles]+ x_spacing/2
    y= y.flatten()[:num_particles]+ y_spacing/2
    positions= np.linalg.matrix_transpose(np.array([x,y]))
    print("init positions: type",type(positions),"size",np.shape(positions))  
    return positions 

In [ ]:
def relative_distnace(
        x_1: NDArray,
        x_2: NDArray,
        box: tuple
) -> NDArray:
    """
    Determines the minimum relative distance over the boundry conditions.

    Parameters:

        x_1: (NDArray)
                -> position Vector of Particle 1
        
        x_2: (NDArray)
                -> position Vector of Particle 2
        
        box: (tuple[float, flaot])
                -> simulation box side length [nm] 

    Return: (tuple[NDArray])
        
        r_min: (NDArray)
                -> vector; nearest neighbour search
        
        d_min: (float)
                -> flaot; bearest neighbour search
    """
    rel6 = np.zeros((len(box),7))
    rel6[:,0] = x_1-x_2 
    rel6[:,1] = x_1-x_2 + np.array([box[0], 0])
    rel6[:,2] = x_1-x_2 + np.array([0, box[1]])
    rel6[:,3] = x_1-x_2 + np.array([box[0]])
    rel6[:,4] = x_1-x_2 - np.array([box[0], 0])
    rel6[:,5] = x_1-x_2 - np.array([0, box[1]])
    rel6[:,6] = x_1-x_2 - np.array([box[0], box[1]])
    d6 = np.linalg.vector_norm(rel6, axis=0)
    min = np.argmin(d6)
    return rel6[:,min], d6[min]

In [59]:
def initiate_distances(positions: NDArray ,box: tuple) -> NDArray:
    distances = np.zeros((len(positions),len(positions)))
    vec_distance= np.zeros((np.shape(positions)[0],np.shape(positions)[1],np.shape(positions)[0],np.shape(positions)[1]))
    print(np.shape(distances))
    print(np.shape(vec_distance))
    for i in range(len(positions)):
        for j in range(i,len(positions)):
            get_rel_distances= relative_distnace(positions[i],positions[j], box=box)
            vec_distance[i,:,j,:], distances[i,j]= get_rel_distances
            vec_distance[j,:,i,:], distances[j,i]= vec_distance[i,:,j,:], distances[i,j] 
    #print(distances)
    distances= np.where(distances == 0, np.inf, distances)
    return vec_distance, distances


if __name__ == '__main__': 
    num_par=25
    box= (10,10)
    pos= initiate_positions_on_grid(num_par,box)
    print(pos[24])
    d_vec, d_val= initiate_distances(pos, box)
    print(len(d_vec[0]),len(d_vec[:,0]), len(d_vec[:,:,0]), len(d_vec[:,:,:,0]))
    print(d_vec[0])
    print(d_vec[:,0])
    #print(d_val) 

x_spacing: 2.0
y_spacing: 2.0
init positions: type <class 'numpy.ndarray'> size (25, 2)
[9. 9.]
(25, 25)
(25, 2, 25, 2)
2 25 25 25
[[[ 0.  0.]
  [-2.  0.]
  [-4.  0.]
  [ 4.  0.]
  [ 2.  0.]
  [ 0. -2.]
  [-2. -2.]
  [-4. -2.]
  [ 4. -2.]
  [ 2. -2.]
  [ 0. -4.]
  [-2. -4.]
  [-4. -4.]
  [ 4. -4.]
  [ 2. -4.]
  [ 0.  4.]
  [-2.  4.]
  [-4.  4.]
  [ 4.  4.]
  [ 2.  4.]
  [ 0.  2.]
  [-2.  2.]
  [-4.  2.]
  [ 4.  2.]
  [ 2.  2.]]

 [[ 0.  0.]
  [-2.  0.]
  [-4.  0.]
  [ 4.  0.]
  [ 2.  0.]
  [ 0. -2.]
  [-2. -2.]
  [-4. -2.]
  [ 4. -2.]
  [ 2. -2.]
  [ 0. -4.]
  [-2. -4.]
  [-4. -4.]
  [ 4. -4.]
  [ 2. -4.]
  [ 0.  4.]
  [-2.  4.]
  [-4.  4.]
  [ 4.  4.]
  [ 2.  4.]
  [ 0.  2.]
  [-2.  2.]
  [-4.  2.]
  [ 4.  2.]
  [ 2.  2.]]]
[[[ 0.  0.]
  [-2.  0.]
  [-4.  0.]
  ...
  [-4.  2.]
  [ 4.  2.]
  [ 2.  2.]]

 [[-2.  0.]
  [ 0.  0.]
  [-2.  0.]
  ...
  [-2.  2.]
  [-4.  2.]
  [ 4.  2.]]

 [[-4.  0.]
  [-2.  0.]
  [ 0.  0.]
  ...
  [ 0.  2.]
  [-2.  2.]
  [-4.  2.]]

 ...

 [[-4.  2.]
  [-2. 

In [ ]:
def montecarlo_step(position:NDArray, step_size:float) -> NDArray:
    dx= np.random.rand(np.shape(position)[0],np.shape(position)[1])
    dx= dx / np.linalg.norm(dx,axis=0) * step_size
    return dx

In [ ]:
def compare_energies(distances:NDArray, dx:NDArray) -> bool:
    energy = lambda r: 9.847044e-3/r**12-6.2647225/r**6
    distance =  np.where(energy(distances)< energy(distances+dx))